**Libraries Installation**

In [ ]:
!pip install gradio
!pip install transformers torch
!pip install accelerate

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Translation**

In [10]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the model and tokenizer
model_name = "VietAI/envit5-translation"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Translation Function
def translate_text(text, source_lang, target_lang):
    prompt = f"Translate the following text from {source_lang} to {target_lang}: {text}"

    # Tokenize and generate output
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)

    with torch.no_grad():
        output = model.generate(**inputs, max_length=256)

    translated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return translated_text

# Saving Model
path = "/content/drive/My Drive/Chatbot_Project/Translator_Model"
tokenizer.save_pretrained(path)
model.save_pretrained(path)
print(f"Model Saved")

Model Saved


**Question-Answering**

In [12]:
import torch
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
import gradio as gr

# Load the model and tokenizer (Public & Free)
model_name = "atharvamundada99/bert-large-question-answering-finetuned-legal"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

# Question Answering Function
def answer_question(question, context):
    inputs = tokenizer(question, context, return_tensors="pt", truncation=True)

    with torch.no_grad():
        outputs = model(**inputs)

    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits) + 1
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))

    return answer if answer.strip() else "Sorry, I couldn't find a relevant answer."

#Saving Model
path="/content/drive/My Drive/Chatbot_Project/QA_Model"
tokenizer.save_pretrained(path)
model.save_pretrained(path)

print(f"Model Saved")

Model Saved


**Text Summarization**

In [11]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
# Load the model and tokenizer (Public & Free)
model_name = "Falconsai/medical_summarization"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
# Summarization Function
def summarize_text(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)
    with torch.no_grad():
        summary_ids = model.generate(**inputs, max_length=150, min_length=50, length_penalty=2.0, num_beams=4)

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Saving Model
path="/content/drive/My Drive/Chatbot_Project/Summarization_Model"
tokenizer.save_pretrained(path)
model.save_pretrained(path)

print(f"Model Saved")

Model Saved


**GUI**

In [14]:
import gradio as gr

# Function to toggle UI visibility based on selected task
def select_task(task):
    return (
        gr.update(visible=(task == "Translation")),
        gr.update(visible=(task == "Question Answering")),
        gr.update(visible=(task == "Summarization")),
    )

# Function to clear inputs and outputs
def clear_fields():
    return "", "", "", ""

def clear_fields_summary():
    return ""

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## Select Task")

    task_buttons = gr.Radio(["Translation", "Question Answering", "Summarization"], label="Choose a task")

    with gr.Group(visible=False) as translation_ui:
        source_lang = gr.Textbox(label="Source Language")
        target_lang = gr.Textbox(label="Target Language")
        text_input = gr.Textbox(label="Enter Text")
        translate_button = gr.Button("Translate")
        translation_output = gr.Textbox(label="Translated Text")

        # Clear button for Translation
        clear_button_t = gr.Button("Clear")
        clear_button_t.click(clear_fields, inputs=[], outputs=[source_lang, target_lang, text_input, translation_output])

        translate_button.click(translate_text, inputs=[text_input, source_lang, target_lang], outputs=translation_output)

    with gr.Group(visible=False) as qa_ui:
        question_input = gr.Textbox(label="Enter Question")
        context_input = gr.Textbox(label="Enter Context")
        answer_button = gr.Button("Get Answer")
        qa_output = gr.Textbox(label="Answer")

        # Clear button for Question Answering
        clear_button_qa = gr.Button("Clear")
        clear_button_qa.click(clear_fields, inputs=[], outputs=[question_input, context_input, qa_output])

        answer_button.click(answer_question, inputs=[question_input, context_input], outputs=qa_output)

    with gr.Group(visible=False) as summarization_ui:
        text_input_summary = gr.Textbox(label="Enter Text")
        summarize_button = gr.Button("Summarize")
        summary_output = gr.Textbox(label="Summary")

        # Clear button for Summarization
        clear_button_s = gr.Button("Clear")
        clear_button_s.click(clear_fields_summary, inputs=[], outputs=[text_input_summary, summary_output])

        summarize_button.click(summarize_text, inputs=[text_input_summary], outputs=summary_output)

    # Link the task selection with visibility updates
    task_buttons.change(select_task, inputs=[task_buttons], outputs=[translation_ui, qa_ui, summarization_ui])

In [ ]:
demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://196cbe66c63c33d535.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
